In [24]:
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, HTML
from sklearn.metrics.pairwise import linear_kernel
import tqdm
import time
from time import strftime, gmtime
import sys
from pprint import pprint
from IPython.display import Markdown

# Warnings
import warnings
from warnings import simplefilter

pd.options.mode.chained_assignment = None
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

Pandas version: 1.4.2
Matplotlib version: 3.5.1
Seaborn version: 0.11.2


In [ ]:
Load pycodestyle(PEP8) :

In [1]:

%load_ext pycodestyle_magic
%pycodestyle_on -- max_line_length 125

La fonction suivante permet d'écrire un titre sous-ligné.

In [25]:
def title(title):
    print("")
    print(title)
    print(len(title) * "-")

In [26]:
get_data = 0
if get_data == 0:
    # Import des données à partir des fichiers csv et stockage des dataframes
    df_orders = pd.read_csv("olist_orders_dataset.csv")
    df_items = pd.read_csv("olist_order_items_dataset.csv")
    df_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
    df_products = pd.read_csv("olist_products_dataset.csv")
    df_payments = pd.read_csv("olist_order_payments_dataset.csv")
    df_customers = pd.read_csv("olist_customers_dataset.csv")
    df_sellers = pd.read_csv("olist_sellers_dataset.csv")
    df_geolocation = pd.read_csv("olist_geolocation_dataset.csv")
    df_categoryname = pd.read_csv("product_category_name_translation.csv")
    %store df_customers
    %store df_geolocation
    %store df_items
    %store df_payments
    %store df_reviews
    %store df_orders
    %store df_products
    %store df_sellers
    %store df_categoryname
elif get_data == 1:  # On rafraîchit les dataframes depuis le store
    %store - r df_customers
    %store - r df_geolocation
    %store - r df_items
    %store - r df_payments
    %store - r df_reviews
    %store - r df_orders
    %store - r df_products
    %store - r df_sellers
    %store - r df_categoryname

Stored 'df_customers' (DataFrame)
Stored 'df_geolocation' (DataFrame)
Stored 'df_items' (DataFrame)
Stored 'df_payments' (DataFrame)
Stored 'df_reviews' (DataFrame)
Stored 'df_orders' (DataFrame)
Stored 'df_products' (DataFrame)
Stored 'df_sellers' (DataFrame)
Stored 'df_categoryname' (DataFrame)


In [19]:
list_dataset = []
list_shape = []
list_variable = []
list_duplicates = []
list_unique = []
list_null = []

dic_data_model = {
    "Dataset": list_dataset,
    "Shape": list_shape,
    "Variable": list_variable,
    "Duplicates": list_duplicates,
    "Unique": list_unique,
    "Null": list_null,
}


def data_model(df, df_name, var, print_=0, n=5):
    list_dataset.append(df_name)
    list_shape.append(df.shape[0])
    list_variable.append(var)
    list_duplicates.append(df.duplicated(subset=[var]).sum())
    list_unique.append(df[var].unique().shape[0])
    list_null.append(df[var].isna().sum().sum())
    df_data_model = pd.DataFrame(dic_data_model)
    if print_ == 1:
        return df_data_model.tail(n)

In [20]:
def data_model_2(df1, df2, df1_name, df2_name, left_on_, right_on_):
    df1_left = df1.merge(
        df2, left_on=left_on_, right_on=right_on_, how="left"
    ).sort_values(right_on_)
    df1_rows = df1.merge(df2, left_on=left_on_, right_on=right_on_, how="left").shape[0]
    df2_rows = df1.merge(df2, left_on=left_on_, right_on=right_on_, how="right").shape[
        0
    ]
    shared_rows = df1.merge(
        df2, left_on=left_on_, right_on=right_on_, how="inner"
    ).shape[0]
    left_rows = df1_rows - shared_rows
    right_rows = df2_rows - shared_rows
    print(
        "Nombre d'enregistrements du dataset\
        %r non renseignés dans le dataset %r: %r"
        % (df1_name, df2_name, left_rows)
    )
    print(
        "Nombre d'enregistrements du dataset\
        %r non renseignés dans le dataset %r: %r"
        % (df2_name, df1_name, right_rows)
    )

5:80: E501 line too long (88 > 79 characters)
6:80: E501 line too long (87 > 79 characters)


# Fusion 1: 

# Commandes / Evaluations

**orders**

In [6]:
df_orders.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


On supprime les variables qui ne nous intéressent pas.

In [7]:
del df_orders["order_status"]
del df_orders["order_approved_at"]
del df_orders["order_delivered_carrier_date"]
del df_orders["order_delivered_customer_date"]
del df_orders["order_estimated_delivery_date"]

In [8]:
data_model(df_orders, "orders", "order_id", 1)

,Dataset,Shape,Variable,Duplicates,Unique,Null
0,orders,99441,order_id,0,99441,0


**reviews**

In [9]:
df_reviews.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [641]:
data_model(df_reviews, "reviews", "order_id", 1)

,Dataset,Shape,Variable,Duplicates,Unique,Null
0,orders,99441,order_id,0,99441,0
1,reviews,99224,order_id,551,98673,0


<IPython.core.display.Javascript object>

**orders et reviews**

On cherche à associer une note ('review_score') unique à chaque commande.

In [642]:
df_reviews_gr = df_reviews.groupby("order_id").agg({"review_score": "mean"})
df_reviews_gr = df_reviews_gr.reset_index()

<IPython.core.display.Javascript object>

In [643]:
data_model(df_reviews_gr, "reviews_gr", "order_id", 1, 3)

,Dataset,Shape,Variable,Duplicates,Unique,Null
0,orders,99441,order_id,0,99441,0
1,reviews,99224,order_id,551,98673,0
2,reviews_gr,98673,order_id,0,98673,0


<IPython.core.display.Javascript object>

**orders et reviews_gr**

In [724]:
data_model_2(df_reviews_gr, df_orders, "reviews_gr", "orders", "order_id", "order_id")

3:2: W291 trailing whitespace
INFO:pycodestyle:3:2: W291 trailing whitespace


<IPython.core.display.Javascript object>

**Fusion orders / reviews**

Chaque commande est maintenant relié à 1 et 1 seul enregistrement du dataset *'reviews_gr'*.

On fusionne donc les datasets *orders* et *reviews_gr* avec une contrainte de correspondance nécessaire (inner join).

In [645]:
df_orders_reviews = df_orders.merge(
    df_reviews_gr, left_on="order_id", right_on="order_id", how="left"
)
df_orders_reviews.info()

<IPython.core.display.Javascript object>

In [646]:
print("Dimensions df_orders_reviews", df_orders_reviews.shape)

<IPython.core.display.Javascript object>

# Clients / Geolocalisation

**Geolocation**

In [647]:
df_geolocation.info()

<IPython.core.display.Javascript object>

In [648]:
data_model(df_geolocation, "geolocation", "geolocation_zip_code_prefix", 1, 1)

,Dataset,Shape,Variable,Duplicates,Unique,Null
3,geolocation,1000163,geolocation_zip_code_prefix,981148,19015,0


<IPython.core.display.Javascript object>

On regroupe alors les enregistrements sur la variable 'geolocation_zip_code_prefix' et on réalise la moyenne des coordonnées géographiques.

In [649]:
df_geolocation_gr_0 = (
    df_geolocation.groupby("geolocation_zip_code_prefix")
    .agg(
        {
            "geolocation_lat": "mean",
            "geolocation_lng": "mean",
            "geolocation_city": "first",
            "geolocation_state": "first",
        }
    )
    .reset_index()
)

<IPython.core.display.Javascript object>

In [650]:
data_model(df_geolocation_gr_0, "geolocation_gr_0", "geolocation_zip_code_prefix", 1, 2)

,Dataset,Shape,Variable,Duplicates,Unique,Null
3,geolocation,1000163,geolocation_zip_code_prefix,981148,19015,0
4,geolocation_gr_0,19015,geolocation_zip_code_prefix,0,19015,0


1:80: E501 line too long (88 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (88 > 79 characters)


<IPython.core.display.Javascript object>

**customers**

In [651]:
df_customers.info()

<IPython.core.display.Javascript object>

Les enregistrements du dataset *customers* sont reliés au dataset *geolocation_gr_0* par les variables 'customer_zip_code_prefix' et 'geolocation_zip_code_prefix'.

In [652]:
del df_customers["customer_city"]
del df_customers["customer_state"]

<IPython.core.display.Javascript object>

In [653]:
data_model(df_customers, "customers", "customer_zip_code_prefix", 1, 2)

,Dataset,Shape,Variable,Duplicates,Unique,Null
4,geolocation_gr_0,19015,geolocation_zip_code_prefix,0,19015,0
5,customers,99441,customer_zip_code_prefix,84447,14994,0


<IPython.core.display.Javascript object>

**customers et geolocation_gr_0**

In [654]:
data_model_2(
    df_customers,
    df_geolocation_gr_0,
    "customers",
    "geolocation_gr_0",
    "customer_zip_code_prefix",
    "geolocation_zip_code_prefix",
)

<IPython.core.display.Javascript object>

**Fusion customers / geolocation_gr_0**

Les références client n'ont pas toutes de correspondance dans le dataset *geolocation_gr_0*.

On fusionne donc les datasets *customers* et *geolocation_gr_0* avec une contrainte de correspondance facultative sur *geolocation_gr_0* (left join).

In [655]:
df_customers_geolocation = df_customers.merge(
    df_geolocation_gr_0,
    left_on="customer_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left",
)

<IPython.core.display.Javascript object>

In [656]:
df_customers_geolocation

,customer_id,customer_unique_id,customer_zip_code_prefix,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,14409.0,-20.498489,-47.396929,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,9790.0,-23.727992,-46.542848,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,1151.0,-23.531642,-46.656289,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,8775.0,-23.499702,-46.185233,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,13056.0,-22.975100,-47.142925,campinas,SP
...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,3937.0,-23.586003,-46.499638,são paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,6764.0,-23.615830,-46.768533,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,60115.0,-3.734569,-38.510534,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,92120.0,-29.949839,-51.168494,canoas,RS


<IPython.core.display.Javascript object>

on supprime les colonns qui ne sont plus nécessaires

In [657]:
del df_customers_geolocation["geolocation_lat"]
del df_customers_geolocation["geolocation_lng"]
del df_customers_geolocation["geolocation_city"]
del df_customers_geolocation["geolocation_state"]

<IPython.core.display.Javascript object>

In [658]:
print("Dimensions df_customers_geolocation", df_customers_geolocation.shape)

<IPython.core.display.Javascript object>

# Commandes-Evaluations/Clients-Geolocalisation

In [659]:
data_model(df_orders_reviews, "orders_reviews", "order_id")
data_model(df_orders_reviews, "orders_reviews", "customer_id", 1, 4)

,Dataset,Shape,Variable,Duplicates,Unique,Null
4,geolocation_gr_0,19015,geolocation_zip_code_prefix,0,19015,0
5,customers,99441,customer_zip_code_prefix,84447,14994,0
6,orders_reviews,99441,order_id,0,99441,0
7,orders_reviews,99441,customer_id,0,99441,0


<IPython.core.display.Javascript object>

La clé du dataset *orders* est la variable 'order_id' qui définit les commandes.

A chaque commande correspond une référence client, la variable 'customer_id'.

On vérifie que chaque commande est associée à une référence client et inversement.

In [660]:
data_model_2(
    df_customers_geolocation,
    df_orders_reviews,
    "customers_geolocation",
    "orders_reviews",
    "customer_id",
    "customer_id",
)

<IPython.core.display.Javascript object>

**Fusion customers_geolocation / orders_reviews**

In [661]:
df_customers_geolocation_orders_reviews = df_customers_geolocation.merge(
    df_orders_reviews, left_on="customer_id", right_on="customer_id", how="inner"
)
print(df_customers_geolocation_orders_reviews.info())
print("")
print(
    "Nombre de valeurs uniques pour 'customer_unique_id':",
    df_customers_geolocation_orders_reviews["customer_unique_id"].unique().shape[0],
)

2:80: E501 line too long (81 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (81 > 79 characters)
8:80: E501 line too long (84 > 79 characters)
INFO:pycodestyle:8:80: E501 line too long (84 > 79 characters)


<IPython.core.display.Javascript object>

In [662]:
print(
    "Dimensions df_customers_geolocation_orders_reviews",
    df_customers_geolocation_orders_reviews.shape,
)

<IPython.core.display.Javascript object>

# Fusion 2: 

# Articles / vendeur

**items**

In [663]:
df_items.info()

<IPython.core.display.Javascript object>

In [664]:
data_model(df_items, "items", "order_id", 1, 1)

,Dataset,Shape,Variable,Duplicates,Unique,Null
8,items,112650,order_id,13984,98666,0


<IPython.core.display.Javascript object>

**Variable - price_total**

On créé la variable 'price_total' qui correspond à la somme des variables 'price' (prix du produit) et 'freight_value' (frais de transport).

In [665]:
df_items["price_total"] = df_items["price"] + df_items["freight_value"]

<IPython.core.display.Javascript object>

On supprime les variables qui ne nous intéressent pas.

In [666]:
del df_items["price"]
del df_items["freight_value"]
del df_items["shipping_limit_date"]

<IPython.core.display.Javascript object>

**Sellers**

In [667]:
df_sellers.info(verbose=True)

<IPython.core.display.Javascript object>

On supprime les variables qui ne nous intéressent pas.

In [668]:
del df_sellers["seller_state"]
del df_sellers["seller_city"]

<IPython.core.display.Javascript object>

**items et sellers**

In [669]:
data_model(df_items, "items", "seller_id")
data_model(df_sellers, "sellers", "seller_id", 1, 2)

,Dataset,Shape,Variable,Duplicates,Unique,Null
9,items,112650,seller_id,109555,3095,0
10,sellers,3095,seller_id,0,3095,0


<IPython.core.display.Javascript object>

In [670]:
data_model_2(df_items, df_sellers, "items", "sellers", "seller_id", "seller_id")

1:80: E501 line too long (80 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (80 > 79 characters)


<IPython.core.display.Javascript object>

**Fusion items / sellers**

Tous les vendeurs ont été référencés au moins une fois dans le dataset *order_items* et chaque enregistrement de *items* a une référence dans *sellers*.

On fusionne donc les datasets *items* et *sellers* avec une contrainte de correspondance nécessaire (inner join).

In [671]:
df_items_sellers = df_items.merge(
    df_sellers, left_on="seller_id", right_on="seller_id", how="inner"
)

<IPython.core.display.Javascript object>

In [672]:
df_items_sellers.info()

<IPython.core.display.Javascript object>

In [673]:
del df_items_sellers["seller_id"]

<IPython.core.display.Javascript object>

# Articles-vendeur/Geolocalisation

**Geolocation**

**nous définissons un nouveau dataset *geolocation_gr* qui ne contient que l'index des préfixes.**

In [674]:
df_geolocation_gr = df_geolocation_gr_0[["geolocation_zip_code_prefix"]]
df_geolocation_gr.head()

,geolocation_zip_code_prefix
0,1001
1,1002
2,1003
3,1004
4,1005


<IPython.core.display.Javascript object>

Les enregistrements du dataset *items_sellers* sont reliés au dataset *geolocation_gr* par les variables 'seller_zip_code_prefix' et 'geolocation_zip_code_prefix'.

**items_sellers et geolocation_gr**

In [675]:
data_model(df_items_sellers, "items_sellers", "seller_zip_code_prefix")
data_model(df_geolocation_gr, "geolocation_gr", "geolocation_zip_code_prefix", 1, 2)

,Dataset,Shape,Variable,Duplicates,Unique,Null
11,items_sellers,112650,seller_zip_code_prefix,110404,2246,0
12,geolocation_gr,19015,geolocation_zip_code_prefix,0,19015,0


2:80: E501 line too long (84 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (84 > 79 characters)


<IPython.core.display.Javascript object>

In [676]:
data_model_2(
    df_items_sellers,
    df_geolocation_gr,
    "items_sellers",
    "geolocation_gr",
    "seller_zip_code_prefix",
    "geolocation_zip_code_prefix",
)

<IPython.core.display.Javascript object>

**Fusion items_sellers / geolocation_gr**

Les lignes de commandes n'ont pas toutes de correspondance dans le dataset *geolocation_gr*.
On fusionne donc les datasets *items_sellers* et *geolocation_gr* avec une contrainte de correspondance facultative sur *geolocation_gr* (left join).

In [677]:
df_items_sellers_geolocation = df_items_sellers.merge(
    df_geolocation_gr,
    left_on="seller_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left",
)

<IPython.core.display.Javascript object>

In [678]:
print("Dimensions df_items_sellers_geolocation", df_items_sellers_geolocation.shape)

1:80: E501 line too long (84 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (84 > 79 characters)


<IPython.core.display.Javascript object>

In [679]:
df_items_sellers_geolocation

,order_id,order_item_id,product_id,price_total,seller_zip_code_prefix,geolocation_zip_code_prefix
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,72.19,27277,27277.0
1,0188777fe321843a18be24a6e9aa1e53,1,436c8d57ff8d4aa254318e9bd9b48c83,65.84,27277,27277.0
2,0191bbb5166d4b9a68df11797d5acfa9,1,ec02a5d380128f7a188e9ce8f3ddd832,153.07,27277,27277.0
3,03054d8a8eefc2981cfad06f58e27979,1,3e31fd1419cc4cd6adad1925e72e3a03,74.06,27277,27277.0
4,0c325ea6925de749e1420be0cf43587c,1,3a135477bf41350a0b1d8afdf616d446,156.64,27277,27277.0
...,...,...,...,...,...,...
112645,fdfeec75aa07c1d2a88b17b8fbfc7b8f,1,11059273f4bc1a293777e98c89807c62,218.13,14402,14402.0
112646,fe68b52db13993f58175fa589125d345,1,f4135cbdece8245560f7be179533797a,225.21,82620,82620.0
112647,fefacc66af859508bf1a7934eab1e97f,1,69c590f7ffc7bf8db97190b6cb6ed62e,6922.21,86026,86026.0
112648,ff701a7c869ad21de22a6994237c8a00,1,5ff4076c0f01eeba4f728c9e3fa2653c,42.34,3124,3124.0


<IPython.core.display.Javascript object>

# Produits / categoryname 

**products**

In [680]:
df_products.info()

<IPython.core.display.Javascript object>

On supprime les variables qui ne nous intéressent pas.

In [681]:
del df_products["product_name_lenght"]
del df_products["product_description_lenght"]
del df_products["product_photos_qty"]
del df_products["product_weight_g"]
del df_products["product_length_cm"]
del df_products["product_height_cm"]
del df_products["product_width_cm"]

<IPython.core.display.Javascript object>

**categoryname**

In [682]:
df_categoryname.info()

<IPython.core.display.Javascript object>

**products et categoryname**


In [683]:
data_model(df_products, "products", "product_id")
data_model(df_products, "products", "product_category_name")
data_model(df_categoryname, "categoryname", "product_category_name", 1, 3)

,Dataset,Shape,Variable,Duplicates,Unique,Null
13,products,32951,product_id,0,32951,0
14,products,32951,product_category_name,32877,74,610
15,categoryname,71,product_category_name,0,71,0


<IPython.core.display.Javascript object>

In [684]:
data_model_2(
    df_products,
    df_categoryname,
    "products",
    "categoryname",
    "product_category_name",
    "product_category_name",
)

<IPython.core.display.Javascript object>

**Fusion products / categoryname**

Il existe plusieurs produits dont la catégorie n'est pas renseignée dans le dataset *categoryname* pour 2 raisons:

La valeur de catégorie du dataset *products* n'existe pas dans le dataset *categoryname*

La valeur de catégorie du dataset *products* est nulle

On fusionne donc les datasets *products* et *categoryname* avec une contrainte de correspondance facultative sur *categoryname* (left join).

In [685]:
df_products_categoryname = df_products.merge(
    df_categoryname,
    left_on="product_category_name",
    right_on="product_category_name",
    how="left",
)
df_products_categoryname.info()

<IPython.core.display.Javascript object>

In [686]:
print("Dimensions df_products_categoryname", df_products_categoryname.shape)

<IPython.core.display.Javascript object>

#  Articles-vendeur-Geolocalisation/ProduIts-categoryname

**items_sellers_geolocation vs products_categoryname**

In [687]:
data_model(df_items_sellers_geolocation, "items_sellers_geolocation", "product_id")
data_model(df_products_categoryname, "products_categoryname", "product_id", 1, 2)

,Dataset,Shape,Variable,Duplicates,Unique,Null
16,items_sellers_geolocation,112650,product_id,79699,32951,0
17,products_categoryname,32951,product_id,0,32951,0


1:80: E501 line too long (83 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (83 > 79 characters)
2:80: E501 line too long (81 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (81 > 79 characters)


<IPython.core.display.Javascript object>

In [688]:
data_model_2(
    df_items_sellers_geolocation,
    df_products_categoryname,
    "items_sellers_geolocation",
    "products_categoryname",
    "product_id",
    "product_id",
)

<IPython.core.display.Javascript object>

**Fusion - items_sellers_geolocation / products_categoryname**

Tous les produits ont été référencés au moins une fois dans le dataset *items_sellers_geolocation* et chaque enregistrement de *items_sellers_geolocation* a une référence dans *products_categoryname*.

On fusionne donc les datasets *order_items* et *products* avec une contrainte de correspondance nécessaire (inner join).

L'objectif est de connaitre le type de produit par commande.

In [689]:
df_items_sellers_geolocation_products_categoryname = df_items_sellers_geolocation.merge(
    df_products_categoryname, left_on="product_id", right_on="product_id", how="inner"
)

df_items_sellers_geolocation_products_categoryname.info()

1:80: E501 line too long (88 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (88 > 79 characters)
2:80: E501 line too long (86 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (86 > 79 characters)


<IPython.core.display.Javascript object>

In [690]:
print(
    "Dimensions df_items_sellers_geolocation_products_categoryname",
    df_items_sellers_geolocation_products_categoryname.shape,
)

<IPython.core.display.Javascript object>

**Montant par commande**

In [691]:
df_items_sellers_geolocation_products_categoryname_gr = df_items_sellers_geolocation_products_categoryname.groupby(
    "order_id"
).agg(
    {"price_total": "sum"}
)

df_items_sellers_geolocation_products_categoryname_gr = (
    df_items_sellers_geolocation_products_categoryname_gr.reset_index()
)

# On renomme les colonnes
df_items_sellers_geolocation_products_categoryname_gr.columns = [
    "order_id",
    "montant_total",
]

1:80: E501 line too long (115 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (115 > 79 characters)


<IPython.core.display.Javascript object>

In [692]:
df_items_sellers_geolocation_products_categoryname_gr.info()

<IPython.core.display.Javascript object>

# Fusion 3: 

# customers_geolocation_orders_reviews / items_sellers_geolocation_products_categoryname_gr

**customers_geolocation_orders_reviews et items_sellers_geolocation_products_categoryname**

In [693]:
data_model(
    df_items_sellers_geolocation_products_categoryname_gr,
    "items_sellers_geolocation_products_categoryname",
    "order_id",
)
data_model(
    df_customers_geolocation_orders_reviews,
    "customers_geolocation_orders_reviews",
    "order_id",
    1,
    2,
)

,Dataset,Shape,Variable,Duplicates,Unique,Null
18,items_sellers_geolocation_products_categoryname,98666,order_id,0,98666,0
19,customers_geolocation_orders_reviews,99441,order_id,0,99441,0


<IPython.core.display.Javascript object>

Le dataset *items_sellers_geolocation_products_categoryname* contient moins de valeurs uniques de 'order_id' que le dataset *customers_geolocation_orders_reviews*.
Il existe en effet un certain nombre de commandes qui n'ont pas de ligne de commande (pas de référence dans le dataset *items_sellers_geolocation_products_categoryname*) donc pas de référence produit.

In [694]:
data_model_2(
    df_customers_geolocation_orders_reviews,
    df_items_sellers_geolocation_products_categoryname_gr,
    "customers_geolocation_orders_reviews",
    "items_sellers_geolocation_products_categoryname_gr",
    "order_id",
    "order_id",
)

<IPython.core.display.Javascript object>

**Fusion customers_geolocation_orders_reviews / items_sellers_geolocation_products_categoryname_gr**

Les informations relatives aux produits et donc aux prix sont nécessaires au projet.

On ne retient donc que les commandes avec des lignes de commande.

On fusionne le dataset *customers_geolocation_orders_reviews* avec le dataset *items_sellers_geolocation_products_categoryname* avec une contrainte de correspondance nécessaire (inner join).

In [695]:
df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname = df_items_sellers_geolocation_products_categoryname_gr.merge(
    df_customers_geolocation_orders_reviews,
    left_on="order_id",
    right_on="order_id",
    how="left",
)

1:80: E501 line too long (150 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (150 > 79 characters)


<IPython.core.display.Javascript object>

In [696]:
print(
    "customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname",
    df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname.shape,
)

2:80: E501 line too long (91 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (91 > 79 characters)
3:80: E501 line too long (98 > 79 characters)
INFO:pycodestyle:3:80: E501 line too long (98 > 79 characters)


<IPython.core.display.Javascript object>

In [697]:
df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname.info()

1:80: E501 line too long (94 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (94 > 79 characters)


<IPython.core.display.Javascript object>

# Fusion 4: 

# Clients-geolocalisation-commandes-evaluations-articles-vendeur-Geolocalisation-ProduIts-categoryname / Paiement

**payments**

In [698]:
df_payments.info()

<IPython.core.display.Javascript object>

**df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname et payments**

In [699]:
data_model(
    df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname,
    "customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname",
    "order_id",
)
data_model(df_payments, "payments", "order_id", 1, 2)

,Dataset,Shape,Variable,Duplicates,Unique,Null
20,customers_geolocation_orders_reviews_items_sel...,98666,order_id,0,98666,0
21,payments,103886,order_id,4446,99440,0


2:80: E501 line too long (92 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (92 > 79 characters)
3:80: E501 line too long (91 > 79 characters)
INFO:pycodestyle:3:80: E501 line too long (91 > 79 characters)


<IPython.core.display.Javascript object>

On cherche à associer une valeur unique de paiement à chaque commande.

Chaque commande peut faire l'objet de plusieurs paiements.

On regroupe les paiements par commande en faisant la somme des paiements.

In [700]:
df_payments_gr = (
    df_payments.groupby("order_id")
    .agg({"payment_value": "sum", "payment_sequential": "count",})
    .sort_values(("payment_value"), ascending=False)
)
df_payments_gr = df_payments_gr.reset_index()
df_payments_gr

,order_id,payment_value,payment_sequential
0,03caa2c082116e1d31e67e9ae3700499,13664.08,1
1,736e1922ae60d0d6a89247b851902527,7274.88,1
2,0812eb902a67711a1cb742b3cdaa65ae,6929.31,1
3,fefacc66af859508bf1a7934eab1e97f,6922.21,1
4,f5136e38d1a14a4dbd87dff67da82701,6726.66,1
...,...,...,...
99435,e8bbc1d69fee39eee4c72cb5c969e39d,10.07,1
99436,f1d5c2e6867fa93ceee9ef9b34a53cbf,9.59,2
99437,c8c528189310eaa44a745b8d9d26908b,0.00,1
99438,00b1cb0320190ca0daa2c88b35206009,0.00,1


3:64: E231 missing whitespace after ','
INFO:pycodestyle:3:64: E231 missing whitespace after ','


<IPython.core.display.Javascript object>

In [701]:
# On renomme les colonnes
df_payments_gr.columns = ["order_id", "payment_sum_per_ord", "payment_sequential"]

2:80: E501 line too long (82 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (82 > 79 characters)


<IPython.core.display.Javascript object>

In [702]:
df_payments_gr.info()

<IPython.core.display.Javascript object>

In [703]:
data_model(df_payments_gr, "payments_gr", "order_id", 1, 3)

,Dataset,Shape,Variable,Duplicates,Unique,Null
20,customers_geolocation_orders_reviews_items_sel...,98666,order_id,0,98666,0
21,payments,103886,order_id,4446,99440,0
22,payments_gr,99440,order_id,0,99440,0


<IPython.core.display.Javascript object>

La dataset *payments_gr* contient maintenant 99440 valeurs uniques pour la variable 'order_id'.

**customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname et payments_gr**

In [704]:
data_model_2(
    df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname,
    df_payments_gr,
    "customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname",
    "payments_gr",
    "order_id",
    "order_id",
)

2:80: E501 line too long (92 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (92 > 79 characters)
4:80: E501 line too long (91 > 79 characters)
INFO:pycodestyle:4:80: E501 line too long (91 > 79 characters)


<IPython.core.display.Javascript object>

**Fusion - customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname / payments_gr**

On fusionne donc les datasets *customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname* et *payments_gr* avec une contrainte de correspondance facultative sur *payments_gr* (left join).

In [705]:
df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments = df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname.merge(
    df_payments_gr, left_on="order_id", right_on="order_id", how="left"
)
df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments.info()

1:80: E501 line too long (193 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (193 > 79 characters)
4:80: E501 line too long (103 > 79 characters)
INFO:pycodestyle:4:80: E501 line too long (103 > 79 characters)


<IPython.core.display.Javascript object>

In [706]:
del df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments[
    "customer_zip_code_prefix"
]
del df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments[
    "geolocation_zip_code_prefix"
]

1:80: E501 line too long (101 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (101 > 79 characters)
4:80: E501 line too long (101 > 79 characters)
INFO:pycodestyle:4:80: E501 line too long (101 > 79 characters)


<IPython.core.display.Javascript object>

In [707]:
print(
    "Dimensions df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments",
    df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments.shape,
)

2:80: E501 line too long (114 > 79 characters)
INFO:pycodestyle:2:80: E501 line too long (114 > 79 characters)
3:80: E501 line too long (107 > 79 characters)
INFO:pycodestyle:3:80: E501 line too long (107 > 79 characters)


<IPython.core.display.Javascript object>

In [708]:
data_commande = df_customers_geolocation_orders_reviews_items_sellers_geolocation_products_categoryname_payments

1:80: E501 line too long (112 > 79 characters)
INFO:pycodestyle:1:80: E501 line too long (112 > 79 characters)


<IPython.core.display.Javascript object>

In [709]:
data_commande

,order_id,montant_total,customer_id,customer_unique_id,order_purchase_timestamp,review_score,payment_sum_per_ord,payment_sequential
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb,2017-09-13 08:59:02,5.0,72.19,1.0
1,00018f77f2f0320c557190d7a144bdd3,259.83,f6dd3ec061db4e3987629fe6b26e5cce,eb28e67c4c0b83846050ddfb8a35d051,2017-04-26 10:53:06,4.0,259.83,1.0
2,000229ec398224ef6ca0657da4fc703e,216.87,6489ae5e4333f3693df5ad4372dab6d3,3818d81c6709e39d06b2738a8d3a2474,2018-01-14 14:33:31,5.0,216.87,1.0
3,00024acbcdf0a6daa1e931b038114c75,25.78,d4eb9395c8c0431ee92fce09860c5a06,af861d436cfc08b2c2ddefd0ba074622,2018-08-08 10:00:35,4.0,25.78,1.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,58dbd0b2d70206bf40e62cd34e84d795,64b576fb70d441e8f1b2d7d446e483c5,2017-02-04 13:57:51,5.0,218.04,1.0
...,...,...,...,...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,343.40,b51593916b4b8e0d6f66f2ae24f2673d,0c9aeda10a71f369396d0c04dce13a64,2018-04-23 13:57:06,5.0,343.40,1.0
98662,fffcd46ef2263f404302a634eb57f7eb,386.53,84c5d4fbaf120aae381fad077416eaa0,0da9fe112eae0c74d3ba1fe16de0988b,2018-07-14 10:26:46,5.0,386.53,1.0
98663,fffce4705a9662cd70adb13d4a31832d,116.85,29309aa813182aaddc9b259e31b870e6,cd79b407828f02fdbba457111c38e4c4,2017-10-23 17:07:56,5.0,116.85,1.0
98664,fffe18544ffabc95dfada21779c9644f,64.71,b5e6afd5a41800fdf401e0272ca74655,eb803377c9315b564bdedad672039306,2017-08-14 23:02:59,5.0,64.71,1.0


<IPython.core.display.Javascript object>

In [710]:
# Export des données nettoyées sans pré-processing
data_commande.to_csv("OLIST_data_commande0_clean.csv")

<IPython.core.display.Javascript object>

**Montant par client**

**On calcule, pour chaque client unique, la somme des montants par commande (variable 'price_per_cust').**

In [711]:
data_client = data_commande.groupby("customer_unique_id").agg(
    {
        "order_id": "count",
        "order_purchase_timestamp": "max",
        "montant_total": "sum",
        "review_score": "mean",
        "payment_sequential": "mean",
    }
)
data_client.columns = [
    "nombre de commande",
    "date de la derniere comande",
    "montant_total_client",
    "satisfaction",
    "payment_sequential",
]
data_client.reset_index(inplace=True)

<IPython.core.display.Javascript object>

In [712]:
data_client

,customer_unique_id,nombre de commande,date de la derniere comande,montant_total_client,satisfaction,payment_sequential
0,0000366f3b9a7992bf8c76cfdf3221e2,1,2018-05-10 10:56:27,141.90,5.0,1.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,2018-05-07 11:11:27,27.19,4.0,1.0
2,0000f46a3911fa3c0805444483337064,1,2017-03-10 21:05:03,86.22,3.0,1.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,2017-10-12 20:29:41,43.62,4.0,1.0
4,0004aac84e0df4da2b147fca70cf8255,1,2017-11-14 19:45:42,196.89,5.0,1.0
...,...,...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,1,2017-06-08 21:00:36,2067.42,5.0,1.0
95416,fffea47cd6d3cc0a88bd621562a9d061,1,2017-12-10 20:07:56,84.58,4.0,1.0
95417,ffff371b4d645b6ecea244b27531430a,1,2017-02-07 15:49:16,112.46,5.0,1.0
95418,ffff5962728ec6157033ef9805bacc48,1,2018-05-02 15:17:41,133.69,5.0,1.0


<IPython.core.display.Javascript object>

In [713]:
# Calculer la moyenne de la colonne 'satisfaction'
satisfaction_mean = data_client["satisfaction"].mean()

# Remplacer les valeurs NaN par la moyenne dans la colonne satisfaction
data_client["satisfaction"] = data_client["satisfaction"].fillna(satisfaction_mean)

# Remplacement des NaN par 1 dans la colonne 'nbr de fois de payement'
data_client["payment_sequential"].fillna(1, inplace=True)

5:80: E501 line too long (83 > 79 characters)
INFO:pycodestyle:5:80: E501 line too long (83 > 79 characters)


<IPython.core.display.Javascript object>

In [714]:
# Export des données nettoyées sans pré-processing
data_client.to_csv("OLIST_data_commande_clean.csv")

<IPython.core.display.Javascript object>

# Création des variables pour la segmentation RFM
La segmentation RFM vise à connaître les comportements d’achat de la clientèle existante et éventuellement à prédire leurs comportements ainsi que celui des prospects. Les trois paramètres retenus sont les suivants :

Récence : la date du dernier achat. On part du principe qu’une personne qui a acheté récemment chez vous a plus de chances de revenir commander chez vous.

Fréquence : le nombre d’achats réalisé sur une période donnée. Plus un client achète régulièrement chez vous, plus il y a de chances pour qu’il achète à nouveau. On analyse ici son niveau de fidélité.

Montant : la somme des achats cumulés sur une période donnée. Les gros acheteurs répondent mieux que les petits. On mesure ici la valeur client.

**Indicateur RFM - Récence**

La récence est la durée entre une date de référence et la dernière date d'achat de chaque client.Habituellement, en marketing, la date de référence est la date de dernière transaction plus 1 jour.

In [715]:
from datetime import datetime

# Conversion des chaînes de caractères en objets datetime
data_commande["order_purchase_timestamp"] = pd.to_datetime(
    data_commande["order_purchase_timestamp"]
)

# Période de commande (dernier achat)
min_date = data_commande["order_purchase_timestamp"].min().strftime("%d/%m/%Y %H:%M")
max_date = data_commande["order_purchase_timestamp"].max().strftime("%d/%m/%Y %H:%M")
print(f"Période d'achat : du {min_date} au {max_date}")

9:80: E501 line too long (85 > 79 characters)
INFO:pycodestyle:9:80: E501 line too long (85 > 79 characters)
10:80: E501 line too long (85 > 79 characters)
INFO:pycodestyle:10:80: E501 line too long (85 > 79 characters)


<IPython.core.display.Javascript object>

In [716]:
import datetime

date_ref = data_commande["order_purchase_timestamp"].max() + datetime.timedelta(days=1)
s_dateref = date_ref.strftime("%d/%m/%Y %Hh%m")
print(f"Data de référence pour la récence : {s_dateref}")

3:80: E501 line too long (87 > 79 characters)
INFO:pycodestyle:3:80: E501 line too long (87 > 79 characters)


<IPython.core.display.Javascript object>

In [717]:
# Alimentation du dictionnaire d'aggrégation pour une ligne de client
# unique
# Ajout de la récence en nombre de jours
dico_aggr_rfm = {"order_purchase_timestamp": lambda x: (date_ref - x.max()).days}
dico_rename_rfm = {"order_purchase_timestamp": "rfm_recence"}

4:80: E501 line too long (81 > 79 characters)
INFO:pycodestyle:4:80: E501 line too long (81 > 79 characters)


<IPython.core.display.Javascript object>

**Indicateur RFM - Fréquence**

La fréquence est 'le nombre de fois où votre client a fait une transaction pendant une période donnée' donc le nombre de commande sur la période de l'étude.

In [718]:
# Alimentation du dictionnaire d'aggrégation pour une ligne de client
# unique
# Ajout de la fréquence en nombre d'achat sur la période
dico_aggr_rfm["order_id"] = "count"
dico_rename_rfm["order_id"] = "rfm_frequence"

<IPython.core.display.Javascript object>

**Indicateur RFM - Montant**

Le montant correspond à la somme ou moyenne des dépenses du client 

In [719]:
# Alimentation du dictionnaire d'aggrégation pour une ligne de client
# unique
# Ajout de la fréquence en nombre d'achat sur la période
dico_aggr_rfm["payment_sum_per_ord"] = "sum"
dico_rename_rfm["payment_sum_per_ord"] = "rfm_montant"

<IPython.core.display.Javascript object>

**Le dataframe RFM**

In [720]:
# Constitution du dataframe de travail RFM
df_rfm = data_commande.groupby("customer_unique_id").agg(dico_aggr_rfm)
df_rfm.rename(columns=dico_rename_rfm, inplace=True)
df_rfm.reset_index()
df_rfm.shape

(95420, 3)

<IPython.core.display.Javascript object>

In [721]:
df_rfm

,rfm_recence,rfm_frequence,rfm_montant
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19
0000f46a3911fa3c0805444483337064,542,1,86.22
0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62
0004aac84e0df4da2b147fca70cf8255,293,1,196.89
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,452,1,2067.42
fffea47cd6d3cc0a88bd621562a9d061,267,1,84.58
ffff371b4d645b6ecea244b27531430a,573,1,112.46


<IPython.core.display.Javascript object>

In [722]:
df_rfm.rfm_frequence.value_counts(normalize=True)

1     0.969472
2     0.028013
3     0.002012
4     0.000304
5     0.000094
6     0.000052
7     0.000031
9     0.000010
16    0.000010
Name: rfm_frequence, dtype: float64

<IPython.core.display.Javascript object>

In [723]:
# Export des données nettoyées sans pré-processing
df_rfm.to_csv("OLIST_data_client_clean.csv")

<IPython.core.display.Javascript object>